## Resnet50

In [6]:
import torch.nn as nn
import torch.nn.functional as F

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(1, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out 
    
def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])

Resnet50 = ResNet50()

import torch
import torchvision
import torchvision.transforms as transforms
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("./PROJECT"), '../..')))
from Models.Util.loadData import create_hanzi_dataloaders


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

train_transform = transforms.Compose(
    [transforms.RandomHorizontalFlip(p=0.5),
     transforms.RandomVerticalFlip(p=0.5),
     transforms.RandomCrop(32, padding=4, padding_mode="reflect"),
     transforms.ToTensor(),
    #  transforms.Normalize([0.5], [0.5])
     ])


batch_size = 32

def get_train_loader(batch_size, transform):
    
    trainloader = create_hanzi_dataloaders(root_dir="../../Dataset/hanzi/processed", batch_size=batch_size,preprocess=train_transform)
    return trainloader
    
    

trainloader = get_train_loader(batch_size, train_transform)


import torch.optim as optim


def get_optimizer(net, lr):
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
    return optimizer


def accuracy(output, target):
    # get the index of the max log-probability
    pred = output.max(1, keepdim=True)[1]
    return pred.eq(target.view_as(pred)).float().mean()

criterion = nn.CrossEntropyLoss()
optimizer = get_optimizer(Resnet50, 0.1)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=60)


cuda:0


In [7]:

def forward_step(net, inputs, labels):
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    return outputs, loss, labels

def train(net, loader, optimizer, max_epoch):
    net.to(device)
    net.train()
    N = len(loader)
    print_interval = (N // 8 // 100 + 1) * 100
    for epoch in range(max_epoch):  # loop over the dataset multiple times

        running_loss = 0.0
        running_acc = 0.0
        for i, data in enumerate(loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            images, labels = data[0].to(device), data[1].to(device) 

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs, loss, labels = forward_step(net, images, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            running_acc += accuracy(outputs, labels).item()
            if (i + 1) % print_interval == 0:
                print('Epoch: [%d / %d], batches: [%d / %d], loss: %.3f, acc: %.2f' %
                      (epoch + 1, max_epoch, i + 1, N, 
                       running_loss / print_interval, 100 * running_acc / print_interval))
                running_loss = 0.0
                running_acc = 0.0
                
        scheduler.step()
        
    print('Finished Training')


train(Resnet50, trainloader, optimizer, 40)

Epoch: [1 / 40], batches: [100 / 147], loss: 6.817, acc: 12.91
Epoch: [2 / 40], batches: [100 / 147], loss: 2.185, acc: 16.19
Epoch: [3 / 40], batches: [100 / 147], loss: 2.066, acc: 20.41
Epoch: [4 / 40], batches: [100 / 147], loss: 1.829, acc: 27.50
Epoch: [5 / 40], batches: [100 / 147], loss: 1.663, acc: 36.50
Epoch: [6 / 40], batches: [100 / 147], loss: 1.447, acc: 46.25
Epoch: [7 / 40], batches: [100 / 147], loss: 1.216, acc: 54.81
Epoch: [8 / 40], batches: [100 / 147], loss: 0.884, acc: 69.38
Epoch: [9 / 40], batches: [100 / 147], loss: 0.740, acc: 73.91
Epoch: [10 / 40], batches: [100 / 147], loss: 0.563, acc: 80.59
Epoch: [11 / 40], batches: [100 / 147], loss: 0.460, acc: 84.44
Epoch: [12 / 40], batches: [100 / 147], loss: 0.435, acc: 85.97
Epoch: [13 / 40], batches: [100 / 147], loss: 0.407, acc: 86.22
Epoch: [14 / 40], batches: [100 / 147], loss: 0.377, acc: 88.31
Epoch: [15 / 40], batches: [100 / 147], loss: 0.314, acc: 89.72
Epoch: [16 / 40], batches: [100 / 147], loss: 0.3

# Save Model

In [8]:
torch.save(Resnet50.state_dict(), './checkpoints/resnet50_ddpm.pth')